In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os

parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.insert(0, parent_dir)

# Getting the data

As always, let's import our last updated data.

In [2]:
from utils import load_datasets

gru_data_folder = os.path.join(parent_dir, "3_preprocessing", "output", "gru")
gru_train_set, gru_val_set, gru_test_set = load_datasets(folder=gru_data_folder)

grubert_data_folder = os.path.join(parent_dir, "3_preprocessing", "output", "grubert")
grubert_train_set, grubert_val_set, grubert_test_set = load_datasets(folder=grubert_data_folder)

# Grid Search  

We will perform a grid search over the parameters that we deem most important, given our resources.  
We derive some information from the grid search for the polarity detection task, which, although different as a task, was performed over the same dataset and can therefore give us the occasion of pruning the search tree a bit.  
In particular, `batch_size=8` and the AdamW `weight_decay=0.10` were assignments that consistently won over other possibilities, so we won't be searching over these two parameters.  
Moreover, we will take some choices a priori based on the idea that, because we need to sacrifice parts of the search space because of limited resources, we at least try to choose the assignments that make more sense based on our two main objectives: regularization and addressing the unbalancedness of our dataset.

## Gru Grid Search  

Here is the list of params and assignements that we will perform the search on:
- `loss_fn`: Binary CrossEntropy and Weighted Focal Loss (our dataset is really uinbalanced so we think it is a good choice to investigate losses that address unbalancedness).
- `lr`: 1e-5 (ask marcello why) and cosine decay, as this was consistently the best annealing strategy for the polarity detection grid search.  
- `label_smoothing`: 0.1, 0.2.
- `scale_grad_by_freq` (see torch.nn.Gru documentation for meaning): True, False.
- `gru_hidden_size`: 128, 256: a hidden size of 128 is small enough to train relatively quickly and often performs well for many text classification tasks.
256 provides more representational capacity if your data has enough examples to support a larger model.
- `num_gru_layers`: 1, 2: A single-layer GRU is often a good baseline for smaller datasets or when resources are limited.
Two layers can capture more complex temporal dependencies in the data and might improve performance if the data benefits from deeper representations.
Going beyond two layers increases training time and risk of overfitting, so staying with [1, 2] is a practical compromise.
- `gru_dropout`: 0.1, 0.2.

In [3]:
from gensim.models import KeyedVectors
import vocabulary
from CustomDataset import GRUDataset

MAX_LEN = 50

# Instantiating vocab
embedding_model_path = os.path.join(parent_dir, "embedding_models","italian_word2vec_100.bin")
embedding_model = KeyedVectors.load_word2vec_format(embedding_model_path, binary=True)

vocab =  vocabulary.Vocabulary(
    dataset=gru_train_set,
    embedding_model=embedding_model,
    tokenizer=vocabulary.BaseTokenizer(),
    embedding_size=100
)
vocab.create_vocabulary(verbose=True)

# Gru Training Dataset
gru_training_dataset = GRUDataset(
    dataframe=gru_train_set,
    vocabulary=vocab,
    max_len=MAX_LEN,
    tokenizer=vocabulary.BaseTokenizer()
)

Importato package text_enrichment.
No cached vocabulary loaded. Building vocabulary from scratch...
[1/5] Tokenizing...
[2/5] Checking for new terms...
	Total new terms: 2830 (16.78%)
[3/5] Merging new terms...
[4/5] Getting word_to_idx and idx_to_word...
[5/5] Building embedding matrix...
Done!


In [11]:
from grid_search import GRUGridSearch
from ModelOps import bce, wfl
import vocabulary
from torch import cuda

device = "cuda" if cuda.is_available() else "cpu"

params = {
    "loss_fn": [bce, wfl],
    "lr": [1e-5, "cosine"],
    "label_smoothing": [0.0, 0.1],
    "num_gru_layers": [1, 2],
    "gru_dropout": [0.1, 0.2],
    "batch_size": [8],
    "scale_grad_by_freq": [False, True], # TODO: i forgot this. relaunch grid search with this :(
    "gru_hidden_size": [128, 256], # TODO: i forgot this. relaunch grid search with this :(
    "tokenizer": [vocabulary.BaseTokenizer()]
}

grid = GRUGridSearch(
    params=params,
    data_folder=gru_data_folder,
    max_len=MAX_LEN,
    device=device,
    parent_dir=parent_dir
)

grid.num_combinations

32

In [12]:
grid.search(30)

Grid Search Progress:   0%|          | 0/32 [00:00<?, ?it/s]

c:\Users\giorg\miniconda3\envs\torch\Lib\site-packages\torch\nn\modules\rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Epochs:   0%|          | 0/30 [00:00<?, ?it/s]

Batches:   0%|          | 0/926 [00:00<?, ?it/s]

Batches:   0%|          | 0/926 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/926 [00:00<?, ?it/s]

Batches:   0%|          | 0/926 [00:00<?, ?it/s]

Batches:   0%|          | 0/926 [00:00<?, ?it/s]

Batches:   0%|          | 0/926 [00:00<?, ?it/s]

Predictions:   0%|          | 0/125 [00:00<?, ?it/s]

In [13]:
grid.get_results()

,loss_fn,lr,batch_size,weight_decay,label_smoothing,optimizer_class,preprocessing,tokenizer,scale_grad_by_freq,gru_hidden_size,num_gru_layers,gru_dropout,text_enrichment,embedding_model,embedding_size,last_save,0/1_f1_score
15,bce,cosine,8,0.1,0.1,AdamW,fundamental,BaseTokenizer,False,32,2,0.2,False,italian_word2vec_100.bin,100,27,0.668153
14,bce,cosine,8,0.1,0.1,AdamW,fundamental,BaseTokenizer,False,32,2,0.1,False,italian_word2vec_100.bin,100,27,0.666238
10,bce,cosine,8,0.1,0.0,AdamW,fundamental,BaseTokenizer,False,32,2,0.1,False,italian_word2vec_100.bin,100,20,0.659827
11,bce,cosine,8,0.1,0.0,AdamW,fundamental,BaseTokenizer,False,32,2,0.2,False,italian_word2vec_100.bin,100,20,0.658219
12,bce,cosine,8,0.1,0.1,AdamW,fundamental,BaseTokenizer,False,32,1,0.1,False,italian_word2vec_100.bin,100,28,0.634492
13,bce,cosine,8,0.1,0.1,AdamW,fundamental,BaseTokenizer,False,32,1,0.2,False,italian_word2vec_100.bin,100,28,0.634492
8,bce,cosine,8,0.1,0.0,AdamW,fundamental,BaseTokenizer,False,32,1,0.1,False,italian_word2vec_100.bin,100,15,0.617102
9,bce,cosine,8,0.1,0.0,AdamW,fundamental,BaseTokenizer,False,32,1,0.2,False,italian_word2vec_100.bin,100,15,0.617102
1,bce,0.00001,8,0.1,0.0,AdamW,fundamental,BaseTokenizer,False,32,1,0.2,False,italian_word2vec_100.bin,100,29,0.588776
0,bce,0.00001,8,0.1,0.0,AdamW,fundamental,BaseTokenizer,False,32,1,0.1,False,italian_word2vec_100.bin,100,29,0.588776
